## 知識情報学第6回演習サンプルプログラム ex6.ipynb
- Programmed by Wu Hongle, 監修　福井健一
- Last updated: 2019/09/02
- Checked with Python 3.8.8, scikit-learn 1.0
- MIT Lisence

## SVMによるBreast Cancerデータの識別
- 入れ子交差検証で最適パラメータを探索

In [46]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import scale

### Breast Cancerデータのロード

In [47]:
df = load_breast_cancer()
X = df.data
y = df.target

# z標準化
X = scale(X)

In [48]:
print(df.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

### 入れ子交差検証でハイパーパラメータを最適化
- 【課題1】探索するパラメータにカーネル関数の追加や範囲を変更して最適パラメータを探してみましょう
    - グリッドサーチパラメータリストの書き方は下記を参照
        - https://scikit-learn.org/stable/modules/grid_search.html#grid-search
    - SVCの可能なパラメータリストは下記を参照
        - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
- 【課題2】Optunaを利用してハイパーパラメータを最適化し，グリッドサーチと比較してみましょう．
    - Optuna: https://optuna.org
    - 使い方は，Code Exmaplesを参照
    - グリッドサーチ同様に入れ子の交差検証を用いること
    - optunaでパラメータの生成範囲指定は下記を参照（関数 suggest_***）
        - https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial
- 【課題3】最適なカーネル関数およびハイパーパラメータ，そこから分かるデータの特徴について考察してみましょう．

In [49]:
# 外側ループのための交差検証用データ生成インスタンス
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

acc_trn_list = []  #外側ループのfold毎の学習データに対するaccuracy格納用
acc_tst_list = []  #外側ループのfold毎のテストデータに対するaccuracy格納用

# グリッドサーチのパラメータリスト
parameters = {'gamma':[0.01, 0.02, 0.05, 0.1, 0,2, 1, 10, 100]} 
# 内側ループでグリッドサーチを行う交差検証インスタンス
gs = GridSearchCV(SVC(), parameters, cv=2) 

k=0
for train_itr, test_itr in kfold.split(X, y):
    # 内側ループのグリッドサーチ
    gs.fit(X[train_itr], y[train_itr])
    print('Fold #{:2d}; Best Parameter: {}, Accuracy: {:.3f}'\
        .format(k+1,gs.best_params_,gs.best_score_))
    acc_trn_list.append(gs.score(X[train_itr],y[train_itr]))
    acc_tst_list.append(gs.score(X[test_itr],y[test_itr]))
    k=k+1

Fold # 1; Best Parameter: {'gamma': 0.02}, Accuracy: 0.969
Fold # 2; Best Parameter: {'gamma': 0.02}, Accuracy: 0.963
Fold # 3; Best Parameter: {'gamma': 0.02}, Accuracy: 0.967
Fold # 4; Best Parameter: {'gamma': 0.02}, Accuracy: 0.961
Fold # 5; Best Parameter: {'gamma': 0.02}, Accuracy: 0.967
Fold # 6; Best Parameter: {'gamma': 0.02}, Accuracy: 0.973
Fold # 7; Best Parameter: {'gamma': 0.02}, Accuracy: 0.961
Fold # 8; Best Parameter: {'gamma': 0.02}, Accuracy: 0.963
Fold # 9; Best Parameter: {'gamma': 0.02}, Accuracy: 0.955
Fold #10; Best Parameter: {'gamma': 0.05}, Accuracy: 0.959


### 平均Accuracy

In [50]:
print('Training data: %1.3f' % np.mean(acc_trn_list))
print('Test data: %1.3f' % np.mean(acc_tst_list))

Training data: 0.986
Test data: 0.974


## 【課題1】探索するパラメータにカーネル関数の追加や範囲を変更して最適パラメータを探してみましょう


In [51]:
# 外側ループのための交差検証用データ生成インスタンス
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

acc_trn_list = []  #外側ループのfold毎の学習データに対するaccuracy格納用
acc_tst_list = []  #外側ループのfold毎のテストデータに対するaccuracy格納用

# グリッドサーチのパラメータリスト
parameters = {
    'kernel': ['linear', 'poly', 'rbf'],  # 線形カーネル、多項式カーネル、RBFカーネル
    'C': [0.01, 0.1, 1, 10, 100],  # ハイパーパラメータC
    'degree': [2, 3, 4],  # 多項式カーネルの次数
    'gamma': [0.01, 0.02, 0.05, 0.1, 0.2, 1, 10, 100]  # RBFカーネルと多項式カーネルのgamma
}

# 内側ループでグリッドサーチを行う交差検証インスタンス
gs = GridSearchCV(SVC(), parameters, cv=2) 

k=0
for train_itr, test_itr in kfold.split(X, y):
    # 内側ループのグリッドサーチ
    gs.fit(X[train_itr], y[train_itr])
    print('Fold #{:2d}; Best Parameter: {}, Accuracy: {:.3f}'\
        .format(k+1,gs.best_params_,gs.best_score_))
    acc_trn_list.append(gs.score(X[train_itr],y[train_itr]))
    acc_tst_list.append(gs.score(X[test_itr],y[test_itr]))
    k=k+1
 
print('Training data: %1.3f' % np.mean(acc_trn_list))
print('Test data: %1.3f' % np.mean(acc_tst_list))    

Fold # 1; Best Parameter: {'C': 10, 'degree': 2, 'gamma': 0.02, 'kernel': 'rbf'}, Accuracy: 0.979
Fold # 2; Best Parameter: {'C': 0.1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.971
Fold # 3; Best Parameter: {'C': 0.1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 4; Best Parameter: {'C': 1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 5; Best Parameter: {'C': 0.1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 6; Best Parameter: {'C': 0.1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.979
Fold # 7; Best Parameter: {'C': 10, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}, Accuracy: 0.979
Fold # 8; Best Parameter: {'C': 0.1, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.975
Fold # 9; Best Parameter: {'C': 10, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}, Accuracy: 0.975
Fold #10; Best Parameter: {'C': 10, 'degree': 2, 'gamma': 0.02, 'kernel': 'rbf'}, Accuracy: 0.97

## 【課題2】Optunaを利用してハイパーパラメータを最適化し，グリッドサーチと比較してみましょう．

In [52]:
import optuna
from sklearn.model_selection import cross_val_score

In [53]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    #  パラメータ    
    C = trial.suggest_float('C', 1e-5, 1e5, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])
    degree = trial.suggest_int('degree', 2, 4)
    gamma = trial.suggest_float('gamma', 1e-5, 1e5, log=True)

    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)

    score = cross_val_score(model, X_train, y_train, cv=2).mean()
    # model.fit(X_train, y_train)
    # accuracy = model.score(X_train, y_train)
    
    return score

acc_trn_list = []  #外側ループのfold毎の学習データに対するaccuracy格納用
acc_tst_list = []  #外側ループのfold毎のテストデータに対するaccuracy格納用
k=0
for train_idx, test_idx in kfold.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=500, show_progress_bar=False)
    
    best_params = study.best_params
    
    C = best_params['C']
    kernel = best_params['kernel']
    degree = best_params['degree']
    gamma = best_params['gamma']

    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
    model.fit(X_train, y_train)
    print('Fold #{:2d}; Best Parameter: {}, Accuracy: {:.3f}'\
        .format(k+1,best_params,study.best_value ))
    acc_trn_list.append(model.score(X_train, y_train))
    acc_tst_list.append(model.score(X_test, y_test))
    k=k+1
  

print('Training data: %1.3f' % np.mean(acc_trn_list))
print('Test data: %1.3f' % np.mean(acc_tst_list)) 

Fold # 1; Best Parameter: {'C': 0.2038673777140091, 'kernel': 'linear', 'degree': 2, 'gamma': 1090.134131787581}, Accuracy: 0.979
Fold # 2; Best Parameter: {'C': 0.10214928199232363, 'kernel': 'linear', 'degree': 3, 'gamma': 0.04919861040734022}, Accuracy: 0.971
Fold # 3; Best Parameter: {'C': 0.09515071835601352, 'kernel': 'linear', 'degree': 2, 'gamma': 0.21036030307772735}, Accuracy: 0.977
Fold # 4; Best Parameter: {'C': 0.997781510424965, 'kernel': 'linear', 'degree': 4, 'gamma': 4.584768221378392e-05}, Accuracy: 0.975
Fold # 5; Best Parameter: {'C': 0.20704921355245395, 'kernel': 'linear', 'degree': 3, 'gamma': 503.9406623657401}, Accuracy: 0.977
Fold # 6; Best Parameter: {'C': 0.06208889629748294, 'kernel': 'linear', 'degree': 2, 'gamma': 95613.28748766422}, Accuracy: 0.982
Fold # 7; Best Parameter: {'C': 14.98752341111002, 'kernel': 'rbf', 'degree': 3, 'gamma': 0.008469594778832}, Accuracy: 0.979
Fold # 8; Best Parameter: {'C': 0.07324636121899472, 'kernel': 'linear', 'degree': 

グリッドサーチを用いてもoptunaを用いてもあまり変化はない。

## 【課題3】最適なカーネル関数およびハイパーパラメータ，そこから分かるデータの特徴について考察してみましょう．

両方の結果で線形カーネルが選択されることが多いことから、データが線形分離可能である可能性が高いと考えられる。しかし、RBFカーネルも一部の場合で選択されており、非線形性も存在する可能性がある。\
パラメータCは幅広い値がとられており、分割データに合わせて調整されていると考えられる。